In [1]:
import numpy as np
import pandas as pd
from scipy.stats.contingency import expected_freq
from scipy.stats import nbinom
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import itertools
import tensorflow as tf
from tensorflow.keras import metrics
import tensorflow.keras as keras
from keras.optimizers import Adam
!pip install tensorflow-addons
!pip install uszipcode
!pip install tzwhere
!pip install astral
import tensorflow_addons as tfa
from google.colab import drive
drive.mount('/content/drive')
TOM_FILES_DIR = "/content/drive/My Drive/Colab Notebooks/data science- transportation research/"
MAX_FILES_DIR = "/content/drive/My Drive/data science- transportation research/"
FILES_DIR = TOM_FILES_DIR

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.1/612.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.5/121.5 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.3 MB/s eta 0:00:00
  Created wheel for atomicwrites: filename=atomicwrites-1.4.1-py2.py3-none-any.whl size=6940 sha256=2bb10327c412eb09c79772b5db0935bd2cc8061406dc783fa52f0d47e6a6d666
  Stored in directory: /root/.cache/pip/wheels/34/07/0b/33b15f68736109f72ea0bb2499521d87312b932620737447a2
Successfully built atomicwrites
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.19
    Uninstalling SQLAlchemy-2.0.19:
      Successfully uninstalled SQLAlchemy-2.0.19
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 43.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tzwhere: filename=tzwh

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


Mounted at /content/drive


In [2]:
final_df = pd.read_csv(f"{FILES_DIR}baseline_3_inter_final_df_2013_2019.csv")
final_df = final_df.drop(columns=["Unnamed: 0"])

In [3]:
y = final_df.astype(np.float64)['COLLISIONS']
X = final_df.astype(np.float64).drop('COLLISIONS', axis=1)
for col in X.columns:
    if np.all((X[col]==1) | (X[col]==0)):
        continue
    X[col] = (X[col] - X[col].mean())/X[col].std()
X["SMALL_STREET"] = ((X["PathTrail"]==1)|(X["Boardwalk"]==1)|(X["StepStreet"]==1)|(X["Driveway"]==1)|\
                    (X["Alley"]==1)).astype(np.float64)
X.drop(columns=["PathTrail","Boardwalk","StepStreet","Driveway","Alley", "Street" ],
       inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=4)

In [4]:
model = tf.keras.Sequential([
tf.keras.layers.Dense(128, activation="relu", input_shape=(X_train.shape[1],)),
tf.keras.layers.Dense(64, activation="relu"),
tf.keras.layers.Dense(32, activation="relu"),
tf.keras.layers.Dense(16, activation="relu"),
tf.keras.layers.Dense(1)

])

learning_rate = 0.001
model.compile(optimizer=Adam(lr=learning_rate), loss='mean_squared_error')


batch_size = 8192
epochs = 30
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1)

# Evaluate the model on the test set
loss = model.evaluate(X_test, y_test, verbose=0)
print("Test Mean Squared Error:", loss)

# Make predictions on the test set and convert back to original scale
y_pred_log = model.predict(X_test)
y_pred = np.exp(y_pred_log) - 1

# Now y_pred contains the predicted counts for the test set
print("Predicted counts:", y_pred)

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/30
103/103 [==============================] - 8s 66ms/step - loss: 0.4563
Epoch 2/30
103/103 [==============================] - 4s 38ms/step - loss: 0.4380
Epoch 3/30
103/103 [==============================] - 4s 35ms/step - loss: 0.4320
Epoch 4/30
103/103 [==============================] - 4s 35ms/step - loss: 0.4297
Epoch 5/30
103/103 [==============================] - 6s 59ms/step - loss: 0.4284
Epoch 6/30
103/103 [==============================] - 4s 36ms/step - loss: 0.4279
Epoch 7/30
103/103 [==============================] - 3s 33ms/step - loss: 0.4276
Epoch 8/30
103/103 [==============================] - 4s 42ms/step - loss: 0.4289
Epoch 9/30
103/103 [==============================] - 5s 52ms/step - loss: 0.4248
Epoch 10/30
103/103 [==============================] - 4s 35ms/step - loss: 0.4237
Epoch 11/30
103/103 [==============================] - 4s 35ms/step - loss: 0.4229
Epoch 12/30
103/103 [==============================] - 5s 49ms/step - loss: 0.4221
Epoch 13/30
1